In [ ]:
import numpy as np
import pandas as pd
import os

# Words workflow

#### Read in data

In [ ]:
data_list = []

with open("data_in/iamHandwriting/ascii/words.txt") as f:
    for line in f:
        if line[0] == "#":
            continue
        line = line.replace("\n", "")
        l_split = line.split(" ", 8)
        
        data_dict = dict()
        data_dict["wordID"] = l_split[0]
        data_dict["segmentation"] = l_split[1]
        data_dict["bin_thresh"] = int(l_split[2])
        data_dict["x_bound"] = int(l_split[3])
        data_dict["y_bound"] = int(l_split[4])
        data_dict["w_bound"] = int(l_split[5])
        data_dict["h_bound"] = int(l_split[6])
        data_dict["grammar"] = l_split[7]
        data_dict["transcription"] = l_split[8]
        data_list.append(data_dict)

data_df = pd.DataFrame(data_list)
data_df = data_df[["wordID", "transcription",
                   "segmentation", "bin_thresh", "x_bound", "y_bound",
                   "w_bound", "h_bound", "grammar"]]

#### Add new columns

In [ ]:
# location columns
data_df["prefix"] = [x.split("-")[0] for x in data_df["wordID"]]
data_df["form"] = ["-".join([x.split("-")[0], x.split("-")[1]])
                            for x in data_df["wordID"]]
local_path = os.getcwd().replace("\\", "/") + "/"
data_df["path"] = local_path + "data_in/iamHandwriting/words/" + data_df["prefix"] + "/" + data_df["form"] + "/" + data_df["wordID"] + ".png"

#### Find freqency of letters

In [ ]:
letters = dict()

for tran in data_df.transcription:
    for l in list(tran):
        if l not in letters:
            letters[l] = 0
        letters[l] += 1
letters = sorted(letters.items(), key = lambda f: f[1], reverse=True)
# print(letters)

#### Get rid of unwanted rows

In [ ]:
w95 = np.percentile(data_df.w_bound, 95)
h95 = np.percentile(data_df.h_bound, 95)
print(w95, h95)

# size columns
import matplotlib.pyplot as plt
plt.subplot(211)
plt.hist(data_df.w_bound)
plt.axvline(x=w95, color="r")
plt.subplot(212)
plt.hist(data_df.h_bound)
plt.axvline(x=h95, color="r")
plt.show()

In [ ]:
# get rid of the really big images
data_df = data_df[np.logical_and(data_df.w_bound < w95, data_df.h_bound < h95)]

# image is broken
data_df = data_df[data_df.wordID != "r06-022-03-05"]

# get only words that are entirely lowercase letters
data_df["lower"] = [all([y.islower() for y in x]) 
                     for x in data_df["transcription"]]

data_df = data_df[data_df["lower"]].loc[:,["path", "transcription"]]

#### Save only lowercase words

In [ ]:
data_df.to_csv("./data/train.csv", sep="\t", index=False)

# Lines workflow

#### Read in data

In [ ]:
data_list = []

with open("data_in/iamHandwriting/ascii/lines.txt") as f:
    for line in f:
        if line[0] == "#":
            continue
        line = line.replace("\n", "")
        l_split = line.split(" ", 8)
        
        data_dict = dict()
        data_dict["lineID"] = l_split[0]
        data_dict["segmentation"] = l_split[1]
        data_dict["bin_thresh"] = int(l_split[2])
        data_dict["n_components"] = int(l_split[3])
        data_dict["x_bound"] = int(l_split[4])
        data_dict["y_bound"] = int(l_split[5])
        data_dict["w_bound"] = int(l_split[6])
        data_dict["h_bound"] = int(l_split[7])
        data_dict["transcription"] = " ".join(l_split[8].split("|"))
        data_list.append(data_dict)

data_df = pd.DataFrame(data_list)
data_df = data_df[["lineID", "transcription",
                   "segmentation", "bin_thresh", "x_bound", "y_bound",
                   "w_bound", "h_bound", "n_components"]]

#### Add new columns

In [ ]:
# location columns
data_df["prefix"] = [x.split("-")[0] for x in data_df["lineID"]]
data_df["form"] = ["-".join([x.split("-")[0], x.split("-")[1]])
                            for x in data_df["lineID"]]
local_path = os.getcwd().replace("\\", "/") + "/data_in/iamHandwriting/lines/"
data_df["path"] = local_path + data_df["prefix"] + "/" + data_df["form"] + "/" + data_df["lineID"] + ".png"

#### Find freqency of letters

In [ ]:
letters = dict()

for tran in data_df.transcription:
    for l in list(tran):
        if l not in letters:
            letters[l] = 0
        letters[l] += 1
letters = sorted(letters.items(), key = lambda f: f[1], reverse=True)
print(letters)

#### Get rid of unwanted rows

In [ ]:
w95 = np.percentile(data_df.w_bound, 95)
h95 = np.percentile(data_df.h_bound, 95)
print(w95, h95)

# size columns
import matplotlib.pyplot as plt
plt.subplot(211)
plt.hist(data_df.w_bound)
plt.axvline(x=w95, color="r")
plt.subplot(212)
plt.hist(data_df.h_bound)
plt.axvline(x=h95, color="r")
plt.show()

In [ ]:
# get rid of the really big images
data_df = data_df[np.logical_and(data_df.w_bound < w95, data_df.h_bound < h95)]

# get only words that are entirely lowercase letters
data_df["lower"] = [all([y.islower() for y in x]) 
                     for x in data_df["transcription"]]

data_df = data_df[["path", "transcription"]]

#### Save only lowercase words

In [ ]:
data_df.to_csv("./data/train.csv", sep="\t", index=False)